To-Do: entire workflow
chemdraw parse -> combinatorial expansion -> conformer generation -> aso description -> post-processing/dimension-reduction/analysis -> plot/output data

1. ChemDraw Parsing

In [1]:
import molli as ml
import molli.visual
import subprocess

/home/colen2/NCSA-workflow-new/molli/molli/visual/__init__.py:10: UserWarning: `py3Dmol` must be installed for molecule visualization. Skipping dependent functions.
  warn(


In [2]:
cores = ml.files.box_cores_test_1
subs = ml.files.box_substituents_test_1

In [15]:
subprocess.run(['molli', 'parse', '--hadd', f'{cores}', '-o', '../../parsed_box/BOX_cores.mlib'])
subprocess.run(['molli', 'parse', '--hadd', f'{subs}', '-o', '../../parsed_box/BOX_subs.mlib'])

100%|██████████| 15/15 [00:00<00:00, 290.03it/s]


CompletedProcess(args=['molli', 'parse', '--hadd', '/home/colen2/NCSA-workflow-new/molli/molli/files/BOX_substituents_test_1.cdxml', '-o', '../../parsed_box/BOX_subs.mlib'], returncode=0)

In [16]:
m_core = ml.MoleculeLibrary('../../parsed_box/BOX_cores.mlib')
print(len(m_core))
m_core[0]

5


Molecule(name='1', formula='C10 H12 Br2 N2 O2 Unknown2')

In [17]:
m_subs = ml.MoleculeLibrary('../../parsed_box/BOX_subs.mlib')
print(len(m_subs))
m_subs[0]

15


Molecule(name='3', formula='C3 H7 Unknown1')

2. Combinatorial Expansion

In [18]:
subprocess.run(
    [
        'molli',
        'combine',
        '../../parsed_box/BOX_cores.mlib',
        '-s',
        '../../parsed_box/BOX_subs.mlib',
        '-j',
        '96', 
        '-o', 
        '../../in_combinatorial/test_combine.mlib', 
        '-a', 
        'A1', 
        '--obopt', 
        'uff',
        '-m',
        'same'
    ]
)


Will create a library of size 75


100%|██████████| 75/75 [00:02<00:00, 28.05it/s]


CompletedProcess(args=['molli', 'combine', '../../parsed_box/BOX_cores.mlib', '-s', '../../parsed_box/BOX_subs.mlib', '-j', '96', '-o', '../../in_combinatorial/test_combine.mlib', '-a', 'A1', '--obopt', 'uff', '-m', 'same'], returncode=0)

In [19]:
combined = ml.MoleculeLibrary('../../in_combinatorial/test_combine.mlib')
print(len(combined))
combined[0]

75


Molecule(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [20]:
combined[42]

Molecule(name='3_6_6', formula='C30 H38 Br2 N2 O2')

3. Conformer Generation

In [22]:
subprocess.run(['molli', 'conformers', '../../in_combinatorial/test_combine.mlib', '-n', '50', '-o', '../../out_conformers1/test4_conformers.mlib', '-t', '-p', '100'])
# maybe change progress bar to increase based on num_rotatable bonds

100%|██████████| 75/75 [02:35<00:00,  2.08s/it]


CompletedProcess(args=['molli', 'conformers', '../../in_combinatorial/test_combine.mlib', '-n', '50', '-o', '../../out_conformers1/test4_conformers.mlib', '-t', '-p', '100'], returncode=0)

In [3]:
clib = ml.ConformerLibrary('../../out_conformers1/test4_conformers.mlib')
print(len(clib))

75


In [5]:
i = 0
for conf in clib:
    i += conf.n_conformers
    # print(conf)
print(str(i) + ' conformers in library')

8028 conformers in library


In [24]:
clib[0][0]

Conformer(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [25]:
clib[0][1]

Conformer(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [26]:
clib[42][0]

Conformer(name='3_6_6', formula='C30 H38 Br2 N2 O2')

In [27]:
clib[42][24]

Conformer(name='3_6_6', formula='C30 H38 Br2 N2 O2')

4. ASO description of molecules

In [29]:
# subprocess.run(['molli', 'grid', '--mlib', '../../out_conformers1/test_conformers.mlib', '-o', 'molli/lib_gen/test_aso'])


In [3]:
subprocess.run(['molli', 'descriptors', '../../out_conformers1/test4_conformers.mlib', '-o', '../../test_aso_data/test_aso.h5'])

grid shape: (6336, 3)
Allocating storage for descriptors


Loading batches of conformers: 0it [00:00, ?it/s]
Submitting calculations: 100%|██████████| 75/75 [00:09<00:00,  6.61it/s]
                                                                        

Gathering calculation results:   0%|          | 0/75 [00:00<?, ?it/s]

Loading batches of conformers: 1it [00:10, 10.23s/it]                


CompletedProcess(args=['molli', 'descriptors', '../../out_conformers1/test4_conformers.mlib', '-o', '../../test_aso_data/test_aso.h5'], returncode=0)

5. post-processing/dimension-reduction/analysis

In [109]:
# subprocess.run(         # DOESN'T QUIT! also check functionality for plotting and pca
#     [                   # should be better way to implement post_processing stuff
#         'molli', 
#         'cluster', 
#         '../../test_aso_data/test_aso.h5', 
#         '-o', 
#         '../../processed_data/test/test2', 
#         '-v',
#         'false',
#         '-c',
#         'false',
#     ]
# )

100%|██████████| 75/75 [00:00<00:00, 5077.28it/s]
/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/threadpoolctl.py:546: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


AAAHHH


/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/threadpoolctl.py:546: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


AAAHHH


/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/threadpoolctl.py:546: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


...

AAAHHH


/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/threadpoolctl.py:546: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


AAAHHH


/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ethangm2/miniconda3/envs/NCSA-workflow-dev/lib/python3.10/site-packages/threadpoolctl.py:546: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


KeyboardInterrupt: 